In [21]:
import time

In [39]:
from llmcoder.analyze.JediAnalyzer import JediAnalyzer
from llmcoder.analyze.MypyAnalyzer import MypyAnalyzer
from llmcoder.analyze.SignatureAnalyzer import SignatureAnalyzer


In [16]:
code ='''import os
import re

from openai import OpenAI

from llmcoder.analyze.Analyzer import Analyzer
from llmcoder.utils import get_openai_key, get_system_prompt, get_system_prompt_dir


class GPTReviewAnalyzer_v1(Analyzer):
    """
    Concise, professional Python code advisor for targeted feedback.

    Parameters
    ----------
    model : str, optional
        The model to use, by default "gpt-3.5-turbo"
    system_prompt : str, optional
        The system prompt to use, by default `2023-12-02_GPTReviewAnalyzer_v4.txt`, created by GPTBuilder
    min_score : int, optional
        The minimum score to pass the review, by default 6

    """
    def __init__(self, model: str = "gpt-3.5-turbo", system_prompt: str | None = None, min_score: int = 6, temperature: float = 0.2):
        """
        Initialize the GPTReviewAnalyzer

        Parameters
        ----------
        model : str, optional
            The model to use, by default "gpt-3.5-turbo"
        system_prompt : str, optional
            The system prompt to use, by default `2023-12-02_GPTReviewAnalyzer_v4.txt`, created by GPTBuilder
        min_score : int, optional
            The minimum score to pass the review, by default 6
        """
        self.model = model

        self.messages: list[dict[str, str]] = []

        self.client = OpenAI(api_key=get_openai_key())

        if system_prompt is None:
            self.system_prompt = get_system_prompt("2023-12-02_GPTReviewAnalyzer_v4.txt")
        elif system_prompt in os.listdir(get_system_prompt_dir()):
            self.system_prompt = get_system_prompt(system_prompt)
        else:
            self.system_prompt = system_prompt

        self.min_score = min_score

        self.temperature = temperature

    def analyze(self, input: str, completion: str, context: dict[str, dict[str, bool | str]] | None = None) -> dict:
        """
        Analyze the input and completion

        Parameters
        ----------
        input : str
            The input to analyze
        completion : str
            The completion to analyze

        Returns
        -------
        dict
            The analysis of the input and completion

        """
        self.messages = []  # Think about it having a memory in the future and running in parallel as a discriminative reviewer with the generative LLMcoder

        self.messages.append({
            "role": "system",
            "content": self.system_prompt
        })'''
completion = '''self.messages.append({
            "role": "user",
            "content": self._prompt_template(input, completion)
        })
        chat_completion = self.client.chat.completions.create(messages=self.messages, model=self.model, temperature=self.temperature) # type: ignore

        self.messages.append({
            "role": "assistant",
            "content": str(chat_completion.choices[0].message.content)
        })

        # Find the score in the message. The score line has the format "SCORE: <score, int{0, 10}>"
        # Define a robust regex to find the score
        score_regex = re.compile(r"SCORE: (\d+)")
        score_match = score_regex.search(self.messages[-1]["content"])

        if score_match is None:
            return {
                "message": self.messages[-1]["content"],
                "pass": None
            }

        return {
            "message": self.messages[-1]["content"],
            "pass": int(score_match.group(1)) >= self.min_score
        }

    @staticmethod
    def _prompt_template(input: str, completion: str) -> str:
        return f"""[INCOMPLETE]{input}[/INCOMPLETE]

[COMPLETE]{input + completion}[/COMPLETE]"""'''

In [17]:

jedi = JediAnalyzer()

In [32]:
import os
import shutil

In [33]:

if os.path.exists("~/.cache/jedi"):
    shutil.rmtree("~/.cache/jedi")

In [36]:
%%timeit
jedi = JediAnalyzer()
jedi.analyze(code, completion)

[JediAnalyzer] name.fullname: <bound method BaseName.docstring of <Name full_name='openai.OpenAI', description='class OpenAI'>>,
name.signature: Type[OpenAI],
name.docstring: OpenAI(*, api_key: str | None=None, organization: str | None=None, base_url: str | httpx.URL | None=None, timeout: Union[float, Timeout, None, NotGiven]=NOT_GIVEN, max_retries: int=DEFAULT_MAX_RETRIES, default_headers: Mapping[str, str] | None=None, default_query: Mapping[str, object] | None=None, http_client: httpx.Client | None=None, _strict_response_validation: bool=False)
[JediAnalyzer] name.fullname: <bound method BaseName.docstring of <Name full_name='llmcoder.analyze.Analyzer.Analyzer', description='class Analyzer'>>,
name.signature: Type[Analyzer],
name.docstring: Analyzer(verbose: bool=False)
[JediAnalyzer] name.fullname: <bound method BaseName.docstring of <Name full_name='llmcoder.utils.get_openai_key', description='def get_openai_key'>>,
name.signature: get_openai_key(key: str = "") -> str,
name.docstr

In [37]:
mypy = MypyAnalyzer()


In [41]:
%%timeit
sig = SignatureAnalyzer()
sig.analyze(code, completion)

58.6 µs ± 2.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
from llmcoder.analyze import MypyAnalyzer, JediAnalyzer

In [ ]:
mypy = MypyAnalyzer(code, completion)

In [ ]:
jedi = JediAnalyzer(code, completion)